한국항공대학교 홈페이지 (https://kau.ac.kr) 및 이하 공지사항 웹사이트는 Robots.txt 파일이 존재하지 않습니다. 따라서 해당 웹 사이트에서 크롤링을 통해 수집을 할 수 있습니다.

In [62]:
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from PIL import Image
import pytesseract
import pyocr.builders
from io import BytesIO

noti : 학사공지
bbs : 일반공지

학사공지

In [67]:
# 웹 드라이버 초기화
driver = webdriver.Chrome('/usr/local/bin/chromedriver')

# 크롤링할 웹사이트 접속
url = 'http://www.kau.ac.kr/web/pages/gc14561b.do'
driver.get(url)

# 결과를 저장할 데이터프레임 초기화
noti_result = pd.DataFrame(columns=['title', 'content', 'date', 'office', 'img'])

# WebDriverWait 객체 생성
wait = WebDriverWait(driver, 10)

img_text = []

try:
    for i in range (1, 21):  # 페이지 반복
        for j in range (1, 11):  # 공지사항 반복
            # 공지사항 제목 요소 검색 및 클릭
            element = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="notiDfTable"]/tr[{j}]/td[2]/a')))
            # 요소로 스크롤
            actions = ActionChains(driver)
            actions.move_to_element(element).perform()
            # 클릭하고 페이지 소스코드 가져오기
            driver.execute_script("arguments[0].click();", element)
            page_source = driver.page_source

            # 제목 텍스트 추출
            title_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="divViewHeader"]/h4')))
            title_text = title_element.text

            # 내용 텍스트 추출
            cont_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="divViewConts"]')))
            cont_text = cont_element.text

            # 날짜 텍스트 추출
            date_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="divViewHeader"]/ul/li[1]')))
            date_text = date_element.text

            # 담당부서 텍스트 추출
            office_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="divViewHeader"]/ul/li[2]')))
            office_text = office_element.text

            try:
                # 이미지 ocr
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                img_element = soup.find_all('img')
                img_tags = img_element[0]
                img_src = img_tags['src']
                img_url = "http://www.kau.ac.kr"+img_src

                if soup.find('img'):
                    # Tesseract OCR 엔진 선택
                    tools = pyocr.get_available_tools()
                    tool = tools[0]

                    response = requests.get(img_url)
                    image = Image.open(BytesIO(response.content))

                    # OCR 수행, 한국어 설정
                    img_text = tool.image_to_string(image, lang="kor", builder=pyocr.builders.TextBuilder(tesseract_layout=6))

                else: # 이미지가 없는 경우
                    img_text = []

            except:
                print(f"예외 발생: 페이지 {i}, 공지사항 {j}")
                pass

            # 결과 데이터프레임에 저장
            n = (i-1)*10 + j-1
            print(n)
            noti_result.at[n, 'title'] = title_text
            noti_result.at[n, 'content'] = cont_text
            noti_result.at[n, 'date'] = date_text
            noti_result.at[n, 'office'] = office_text
            noti_result.at[n, 'img'] = img_text

            # 이전 페이지(공지사항)로 이동
            driver.back()
            time.sleep(0.3)

        # 다음 (공지사항)페이지로 이동하기 위헤 페이지 번호 클릭
        page_element = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="divPagination"]/a[3]')))
        driver.execute_script("arguments[0].click();", page_element)
        page_source = driver.page_source
        time.sleep(0.5)

# 예외 발생 시 계속 진행
except Exception as e:
    print("예외 발생 : ", e)
    pass

# 웹 드라이버 종료
driver.quit()

noti_result.to_csv('noti_result.csv')

/var/folders/gg/ngckf6_17j36ymx3rd1_jrtc0000gn/T/ipykernel_19786/730564539.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/usr/local/bin/chromedriver')


0
1
예외 발생: 페이지 1, 공지사항 3
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
예외 발생: 페이지 2, 공지사항 10
19
20
21
22
23
예외 발생: 페이지 3, 공지사항 5
24
25
26
예외 발생: 페이지 3, 공지사항 8
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
예외 발생: 페이지 5, 공지사항 6
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
예외 발생: 페이지 8, 공지사항 2
71
예외 발생: 페이지 8, 공지사항 3
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
예외 발생: 페이지 9, 공지사항 8
87
예외 발생: 페이지 9, 공지사항 9
88
89
90
91
92
93
예외 발생: 페이지 10, 공지사항 5
94
예외 발생: 페이지 10, 공지사항 6
95
96
97
98
99
100
101
예외 발생: 페이지 11, 공지사항 3
102
103
104
105
106
107
108
109
110
111
112
113
114
예외 발생: 페이지 12, 공지사항 6
115
116
117
118
119
120
121
122
123
124
125
예외 발생: 페이지 13, 공지사항 7
126
127
128
129
130
131
132
예외 발생: 페이지 14, 공지사항 4
133
134
135
136
137
138
139
140
141
예외 발생: 페이지 15, 공지사항 3
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
예외 발생: 페이지 16, 공지사항 8
157
예외 발생: 페이지 16, 공지사항 9
158
159
160
예외 발생: 페이지 17, 공지사항 2
161
예외 발생: 페이지 17, 공지사항 3
162
예외 발생: 페이지 17, 공지사항 4
163


일반공지

In [71]:
# 웹 드라이버 초기화
driver = webdriver.Chrome('/usr/local/bin/chromedriver')

# 크롤링할 웹사이트 접속
url = 'http://www.kau.ac.kr/web/pages/gc32172b.do'
driver.get(url)

# 결과를 저장할 데이터프레임 초기화
bbs_result = pd.DataFrame(columns=['title', 'content', 'date', 'office', 'img'])

# WebDriverWait 객체 생성
wait = WebDriverWait(driver, 10)
count = 0

try:
    for i in range (1, 42):  # 페이지 반복
        for j in range (1, 11):  # 공지사항 반복
            try:
                # 공지사항 제목 요소 검색 및 클릭
                element = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="bbsDfTable"]/tr[{j}]/td[2]/a')))
                # 요소로 스크롤
                actions = ActionChains(driver)
                actions.move_to_element(element).perform()
                # 클릭하고 페이지 소스코드 가져오기
                driver.execute_script("arguments[0].click();", element)
                page_source = driver.page_source

                # 제목 텍스트 추출
                title_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="divViewHeader"]/h4')))
                title_text = title_element.text

                # 내용 텍스트 추출
                cont_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="divViewConts"]')))
                cont_text = cont_element.text

                # 날짜 텍스트 추출
                date_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="divViewHeader"]/ul/li[1]')))
                date_text = date_element.text

                # 담당부서 텍스트 추출
                office_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="divViewHeader"]/ul/li[2]')))
                office_text = office_element.text

                try:
                    # 이미지 ocr
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    img_element = soup.find_all('img')
                    img_tags = img_element[0]
                    img_src = img_tags['src']
                    img_url = "http://www.kau.ac.kr"+img_src

                    if soup.find('img'):
                        # Tesseract OCR 엔진 선택
                        tools = pyocr.get_available_tools()
                        tool = tools[0]

                        response = requests.get(img_url)
                        image = Image.open(BytesIO(response.content))
                        time.sleep(0.3)

                        # OCR 수행, 한국어 설정
                        img_text = tool.image_to_string(image, lang="kor", builder=pyocr.builders.TextBuilder(tesseract_layout=6))

                    else: # 이미지가 없는 경우
                        img_text = []

                except:
                    print(f"예외 발생: 페이지 {i},{j}번째 공지사항, ", count, "번째 예외")
                    count += 1
                    pass

                # 결과 데이터프레임에 저장
                n = (i-1)*10 + j-1
                print(n)
                bbs_result.at[n, 'title'] = title_text
                bbs_result.at[n, 'content'] = cont_text
                bbs_result.at[n, 'date'] = date_text
                bbs_result.at[n, 'office'] = office_text
                bbs_result.at[n, 'img'] = img_text

                # 이전 페이지(공지사항)로 이동
                driver.back()
                time.sleep(0.5)

            except:
                print(f"예외 발생: 페이지 {i},{j}번째 공지사항, ", count, "번째 예외")
                count += 1
                pass

        try:
            # 다음 (공지사항)페이지로 이동하기 위헤 페이지 번호 클릭
            page_element = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="divPagination"]/a[3]')))
            driver.execute_script("arguments[0].click();", page_element)
            page_source = driver.page_source
            time.sleep(0.5)

        except:
            print(f"예외 발생: 페이지 {i}에서 넘어갈 때 발생.", count, "번째 예외")
            count += 1
            pass

    # 결과 출력
    print(bbs_result)

# 예외 발생 시 계속 진행
except Exception as e:
    print("예외 발생 : ", e)
    pass

# 웹 드라이버 종료
driver.quit()

bbs_result.to_csv('bbs_result.csv')

/var/folders/gg/ngckf6_17j36ymx3rd1_jrtc0000gn/T/ipykernel_19786/1496829674.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/usr/local/bin/chromedriver')


0
1
2
3
4
5
6
7
8
예외 발생: 페이지 1,10번째 공지사항,  0 번째 예외
9
예외 발생: 페이지 2,1번째 공지사항,  1 번째 예외
10
11
12
13
14
15
예외 발생: 페이지 2,7번째 공지사항,  2 번째 예외
16
17
18
19
예외 발생: 페이지 3,1번째 공지사항,  3 번째 예외
20
21
22
예외 발생: 페이지 3,4번째 공지사항,  4 번째 예외
23
24
25
26
27
28
29
30
31
예외 발생: 페이지 4,3번째 공지사항,  5 번째 예외
32
33
34
35
예외 발생: 페이지 4,7번째 공지사항,  6 번째 예외
36
37
38
39
예외 발생: 페이지 5,1번째 공지사항,  7 번째 예외
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
예외 발생: 페이지 6,10번째 공지사항,  8 번째 예외
59
예외 발생: 페이지 7,1번째 공지사항,  9 번째 예외
60
61
62
63
64
65
예외 발생: 페이지 7,7번째 공지사항,  10 번째 예외
66
67
68
69
예외 발생: 페이지 8,1번째 공지사항,  11 번째 예외
70
71
72
예외 발생: 페이지 8,4번째 공지사항,  12 번째 예외
73
74
75
76
77
78
79
80
81
예외 발생: 페이지 9,3번째 공지사항,  13 번째 예외
82
83
84
85
예외 발생: 페이지 9,7번째 공지사항,  14 번째 예외
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
예외 발생: 페이지 11,7번째 공지사항,  15 번째 예외
106
107
108
109
예외 발생: 페이지 12,1번째 공지사항,  16 번째 예외
110
111
112
예외 발생: 페이지 12,4번째 공지사항,  17 번째 예외
113
114
115
116
117
118
119
예외 발생: 페이지 13,1번째 공지사항,  18 번째 예외
120
121
12